In [ ]:
!rm -rf tensorflow-experiment
!git clone https://github.com/fadamsyah/tensorflow-experiment.git

In [ ]:
cd tensorflow-experiment/image_classification

In [ ]:
pwd

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
import shutil
import sys

from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import datasets
from utils.model import AugLayer, VGG, EfficientNetB0, CheckpointCallback, CustomLRDecay, plot_history

from tensorflow.keras.applications.efficientnet import preprocess_input

from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Add
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Dropout
import cv2

import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_train, X_test = list(X_train), list(X_test)

In [ ]:
BATCH_SIZE = 1024
EPOCHS = 50
PATH_CHECKPOINT = './CIFAR10/checkpoint'
PATH_CSV = './CIFAR10/training.log'
LR_PROFILE = [0.1, 0.01, 0.005, 0.001]
LR_EPOCHS = [15, 30, 40, EPOCHS]

AUG_RESIZE= (128,128)
AUG_CONTRAST = 0.15
AUG_CROP = None
AUG_FLIP = 'horizontal'
AUG_ROT = 45.
AUG_TRANSLATION = (0.15, 0.15)
AUG_ZOOM = 0.15

shutil.rmtree(PATH_CHECKPOINT, ignore_errors=True)
os.makedirs(PATH_CHECKPOINT)

In [ ]:
aug_layer = AugLayer(AUG_RESIZE, None, AUG_CONTRAST,
                     AUG_CROP, AUG_FLIP, AUG_ROT,
                     AUG_TRANSLATION, AUG_ZOOM)

model = EfficientNetB0((32,32,3), 10, aug_layer, "imagenet", False)

model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9, nesterov=True), loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
callbacks = [#CheckpointCallback(PATH_CHECKPOINT),
             CustomLRDecay(LR_PROFILE, LR_EPOCHS),
                          CSVLogger(PATH_CSV)]

history = model.fit(X_train, y_train,
                    steps_per_epoch = math.ceil(X_train.shape[0] / BATCH_SIZE),
                    validation_data=(X_test, y_test),
                    validation_steps = math.ceil(X_test.shape[0] / BATCH_SIZE),
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    workers=4)

In [ ]:
model.evaluate(X_test, y_test)